# Moving Average Methods - Solution

This notebook provides a complete solution for implementing moving average methods at TechGear, including:
1. Simple Moving Average (SMA)
2. Weighted Moving Average (WMA)
3. Window Size Selection
4. Forecast Accuracy Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils.testing.forecasting_tests import check_moving_average, check_forecast_accuracy

# Set plotting style
plt.style.use('seaborn')
sns.set_palette('husl')

ModuleNotFoundError: No module named 'utils'

## Part 1: Simple Moving Average

First, we'll implement simple moving average with different window sizes.

In [ ]:
def simple_moving_average(data, window):
    """Calculate simple moving average.
    
    Args:
        data (array): Input time series data
        window (int): Window size for moving average
        
    Returns:
        array: Moving average values (same length as input)
    """
    result = np.zeros_like(data)
    result[:window-1] = np.nan  # First window-1 values can't be calculated
    
    for i in range(window-1, len(data)):
        result[i] = np.mean(data[i-window+1:i+1])
        
    return result

# Sample data: Daily laptop sales
sales_data = np.array([45, 52, 48, 58, 50, 42, 55, 53, 49, 51, 47, 54, 50, 43, 56])

# Calculate moving averages
ma_3 = simple_moving_average(sales_data, 3)
ma_5 = simple_moving_average(sales_data, 5)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(sales_data, marker='o', label='Actual Data')
plt.plot(ma_3, marker='s', label='3-Day MA')
plt.plot(ma_5, marker='^', label='5-Day MA')
plt.title('Simple Moving Average - Laptop Sales')
plt.xlabel('Day')
plt.ylabel('Sales')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calculate forecast accuracy (excluding NaN values)
mse_3 = np.nanmean((sales_data[2:] - ma_3[2:])**2)
mse_5 = np.nanmean((sales_data[4:] - ma_5[4:])**2)

print("Forecast Accuracy:")
print(f"MSE (3-Day MA): {mse_3:.2f}")
print(f"MSE (5-Day MA): {mse_5:.2f}")

# Test moving average implementation
if check_moving_average(sales_data, ma_3, 3):
    print("✓ Simple moving average correctly implemented!")
else:
    print("✗ Check your moving average implementation")

### Analysis of Window Size Selection

1. **Effect of Window Size**:
   - Smaller window (3 days): More responsive to changes, less smoothing
   - Larger window (5 days): More smoothing, slower response

2. **Trade-offs**:
   - Small window: Better for detecting quick changes, but more sensitive to noise
   - Large window: Better for identifying trends, but may miss important changes

3. **Window Size Selection**:
   - Consider business cycle (e.g., weekly patterns suggest 7-day window)
   - Balance between noise reduction and responsiveness
   - Use MSE to compare different windows

## Part 2: Weighted Moving Average

Now we'll implement weighted moving average to give more importance to recent data.

In [ ]:
def weighted_moving_average(data, weights):
    """Calculate weighted moving average.
    
    Args:
        data (array): Input time series data
        weights (array): Weights for moving average (should sum to 1)
        
    Returns:
        array: Weighted moving average values
    """
    window = len(weights)
    result = np.zeros_like(data)
    result[:window-1] = np.nan
    
    for i in range(window-1, len(data)):
        result[i] = np.sum(data[i-window+1:i+1] * weights)
        
    return result

# Define weights (most recent first)
weights_3day = np.array([0.5, 0.3, 0.2])
weights_5day = np.array([0.3, 0.25, 0.2, 0.15, 0.1])

# Calculate weighted moving averages
wma_3 = weighted_moving_average(sales_data, weights_3day)
wma_5 = weighted_moving_average(sales_data, weights_5day)

# Plot comparison
plt.figure(figsize=(12, 6))
plt.plot(sales_data, marker='o', label='Actual Data')
plt.plot(ma_3, '--', label='3-Day SMA')
plt.plot(wma_3, marker='s', label='3-Day WMA')
plt.plot(ma_5, '--', label='5-Day SMA')
plt.plot(wma_5, marker='^', label='5-Day WMA')

plt.title('Simple vs Weighted Moving Average')
plt.xlabel('Day')
plt.ylabel('Sales')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calculate forecast accuracy for weighted MA
mse_wma_3 = np.nanmean((sales_data[2:] - wma_3[2:])**2)
mse_wma_5 = np.nanmean((sales_data[4:] - wma_5[4:])**2)

print("\nForecast Accuracy Comparison:")
print("3-Day Window:")
print(f"MSE (Simple MA): {mse_3:.2f}")
print(f"MSE (Weighted MA): {mse_wma_3:.2f}")
print("\n5-Day Window:")
print(f"MSE (Simple MA): {mse_5:.2f}")
print(f"MSE (Weighted MA): {mse_wma_5:.2f}")

# Test forecast accuracy
if check_forecast_accuracy({'wma_3': mse_wma_3, 'wma_5': mse_wma_5}):
    print("✓ Forecast accuracy correctly calculated!")
else:
    print("✗ Check your forecast accuracy calculations")

### Analysis of Results

1. **Effect of Weights**:
   - Higher weights on recent data: More responsive to recent changes
   - Lower weights on older data: Reduces impact of outliers

2. **When to Use Weighted MA**:
   - Recent data is more relevant (e.g., changing market conditions)
   - Need faster response to changes
   - Seasonal patterns are present

3. **Weight Selection**:
   - Weights should sum to 1
   - Higher weights for recent data (e.g., exponential decay)
   - Consider business context (e.g., weekly cycles)

4. **Comparison with Simple MA**:
   - WMA typically more responsive to changes
   - WMA better for data with trends
   - SMA better for stable, noisy data